In [2]:
%pip install matplotlib seaborn

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [4]:
# === ALL-IN-ONE MODEL TRAINING SCRIPT ===
# This single cell imports all libraries, defines paths, loads data,
# trains both models, and saves them to the correct folder.

import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
import joblib
import os
import sys
from pathlib import Path

print("--- Starting Model Training ---")

# --- 1. SETUP AND PATHS ---
try:
    # This robustly finds the notebook's directory
    notebook_dir = Path(os.path.abspath(''))
    print(f"Notebook directory (where this code is running): {notebook_dir}")

    # Go up one level to the main project directory
    project_dir = notebook_dir.parent
    print(f"Main project directory (ev-anomaly-detection): {project_dir}")

    # Define the correct paths for your data and models folders
    data_path = project_dir / 'data' / 'ev_charging_data.csv'
    models_dir = project_dir / 'models'

    # Create the 'models' directory if it doesn't exist, just in case
    os.makedirs(models_dir, exist_ok=True)
    print(f"Ensured 'models' directory exists at: {models_dir}")

except Exception as e:
    print(f"🚨 ERROR setting up paths: {e}")
    raise

# --- 2. LOAD DATA ---
try:
    df = pd.read_csv(data_path, parse_dates=['start_time', 'end_time'])
    print(f"✅ Successfully loaded data from: {data_path}")
except FileNotFoundError:
    print(f"🚨 ERROR: Data file not found at {data_path}. Please run `generate_data.py` first.")
    raise
except Exception as e:
    print(f"🚨 ERROR loading data: {e}")
    raise


# --- 3. TRAIN & SAVE DOS MODEL ---
print("\n--- [1/2] Training DoS Model ---")
try:
    # Select features and scale
    dos_features = ['cpu_usage_percent', 'packets_per_sec']
    X_dos = df[dos_features]
    scaler_dos = StandardScaler()
    X_dos_scaled = scaler_dos.fit_transform(X_dos)

    # Train the model (using 0.16 for 16% contamination from our data script)
    dos_model = IsolationForest(n_estimators=100, contamination=0.16, random_state=42)
    dos_model.fit(X_dos_scaled)
    print("DoS model training complete.")

    # Save the model and scaler
    dos_model_path = models_dir / 'dos_model.pkl'
    dos_scaler_path = models_dir / 'dos_scaler.pkl'
    joblib.dump(dos_model, dos_model_path)
    joblib.dump(scaler_dos, dos_scaler_path)
    
    print(f"✅ DoS model successfully saved to: {dos_model_path}")
    print(f"✅ DoS scaler successfully saved to: {dos_scaler_path}")

except Exception as e:
    print(f"🚨 ERROR during DoS model training/saving: {e}")


# --- 4. TRAIN & SAVE FRAUD MODEL ---
print("\n--- [2/2] Training Fraud Model ---")
try:
    # Select features and scale
    fraud_features = ['energy_kWh', 'amount_INR']
    X_fraud = df[fraud_features]
    scaler_fraud = StandardScaler()
    X_fraud_scaled = scaler_fraud.fit_transform(X_fraud)

    # Train the model
    fraud_model = IsolationForest(n_estimators=100, contamination=0.16, random_state=42)
    fraud_model.fit(X_fraud_scaled)
    print("Fraud model training complete.")

    # Save the model and scaler
    fraud_model_path = models_dir / 'fraud_model.pkl'
    fraud_scaler_path = models_dir / 'fraud_scaler.pkl'
    joblib.dump(fraud_model, fraud_model_path)
    joblib.dump(scaler_fraud, fraud_scaler_path)
    
    print(f"✅ Fraud model successfully saved to: {fraud_model_path}")
    print(f"✅ Fraud scaler successfully saved to: {fraud_scaler_path}")

except Exception as e:
    print(f"🚨 ERROR during Fraud model training/saving: {e}")

print("\n--- Model Training Script Finished. You can now start your backend server. ---")



--- Starting Model Training ---
Notebook directory (where this code is running): c:\ev-anamoly-detection\ev-anomaly-detection\notebooks
Main project directory (ev-anomaly-detection): c:\ev-anamoly-detection\ev-anomaly-detection
Ensured 'models' directory exists at: c:\ev-anamoly-detection\ev-anomaly-detection\models
✅ Successfully loaded data from: c:\ev-anamoly-detection\ev-anomaly-detection\data\ev_charging_data.csv

--- [1/2] Training DoS Model ---
DoS model training complete.
✅ DoS model successfully saved to: c:\ev-anamoly-detection\ev-anomaly-detection\models\dos_model.pkl
✅ DoS scaler successfully saved to: c:\ev-anamoly-detection\ev-anomaly-detection\models\dos_scaler.pkl

--- [2/2] Training Fraud Model ---
Fraud model training complete.
✅ Fraud model successfully saved to: c:\ev-anamoly-detection\ev-anomaly-detection\models\fraud_model.pkl
✅ Fraud scaler successfully saved to: c:\ev-anamoly-detection\ev-anomaly-detection\models\fraud_scaler.pkl

--- Model Training Script Finis